<a href="https://colab.research.google.com/github/xhdixhfl/DeepLearning-Project/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.cognex.com/ko-kr/blogs/deep-learning/research/deep-learning-segmentation-finding-marking-dogs-cats-gcn

In [ ]:
import torch  

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('There are %d GPU available.' % torch.cuda.device_count())
  print('We will use the GPU : ', torch.cuda.get_device_name(0))
else:
  device = torch.device("CPU")
  print('No GPU. using CPU')

In [ ]:
def read_dt(data_dir, image_size, no_label = True):
  """
    Segmentation문제를 위해 데이터를 전처리하고 로드.
    :param data_dir: image 및 mask 데이터가 저장된 root 경로
    :image_size: tuple, 크롭 및 패딩을 위해 지정된 이미지 사이즈
    :no_label: bool, 레이블을 로드할 지 여부
    :return: X_set: np.ndarray, shape: (N, H, W, C).
             y_set: np.ndarray, shape: (N, H, W, num_classes (include background)).
    """
    im_path = []
    for x in EXT:
      im_paths.extend(glob.glob(os.path.join(data_dir, 'images', '*.{}'.format(x))))
      imgs = []
      labels = []
      for im_path in im_paths:
        # 이미지 로드
        im_name = os.path.splitext(os.path.basename(im_path))[0]
        im = cv2.imread(im_path)
        im = crop_shape(im, image_size)
        im = padding(im, image_size)
        imgs.append(im)

        if no_label:
          labels.append(0)
          continue

        # 마스크 로드
        mask_path = os.path.join(data_dir, 'masks', '{}.png'.format(im_name))
        mask = cv2.imread(mask_path)
        mask = crop_shape(mask, image_size)
        mask = padding(mask, image_size, fill = 2)

        label = np.zeros((image_size[0],image_size[1], 3), dtype=np.float32)
        label.fill(-1)   # Pixel annotations 1:Foreground, 2:Background, 3:Unknown
        idx = np.where(mask == 2)
        label[idx[0], idx[1], :] = [1,0,0]

        idx = np.where(mask == 1)
        if im_name[0].isupper():
          label[idx[0], idx[1], :] = [0,1,0]
        else:
          label[idx[0], idx[1], :] = [0,0,1]
          labels.append(label)

      X_set = np.array(imgs, dtype = np.float32)
      y_set = np.array(labels, dtype = np.float32)

      return X_set, y_set

      
